In [11]:
import arcpy

# Paths to your layers
tmk_layer = r"C:\Users\Sea Grant\Desktop\WW_Overlay\OahuTMK\OahuTMK.shp"  # Update with your TMK layer path
sewer_main_layer = r"C:\Users\Sea Grant\Desktop\WW_Overlay\OahuSewerMains\Sewer_Main.shp"  # Update with your Sewer Main layer path
cesspool_layer = r"C:\Users\Sea Grant\Desktop\WW_Overlay\OahuOSDS\On-site_Sewage_Disposal_Systems_-_Oahu.shp"  # Update with your Cesspool layer path



In [16]:
# Add fields to the TMK layer
fields_to_add = [("Mains", "TEXT"), ("Cesspools", "TEXT")]
for field_name, field_type in fields_to_add:
    if field_name not in [f.name for f in arcpy.ListFields(tmk_layer)]:
        arcpy.AddField_management(tmk_layer, field_name, field_type, field_length=3)

In [18]:
# Create temporary layers
arcpy.MakeFeatureLayer_management(tmk_layer, "TMK_Layer2")
arcpy.MakeFeatureLayer_management(sewer_main_layer, "SewerMain_Layer2")
arcpy.MakeFeatureLayer_management(cesspool_layer, "Cesspool_Layer2")

# Initialize an update cursor to update the "Mains" and "Cesspools" fields
with arcpy.da.UpdateCursor(tmk_layer, ["OID@", "Mains", "Cesspools"]) as cursor:
    for row in cursor:
        oid = row[0]
        
        # Create a query to select the current TMK
        tmk_query = f"OBJECTID = {oid}"
        arcpy.SelectLayerByAttribute_management("TMK_Layer2", "NEW_SELECTION", tmk_query)
        
        # Check for intersection with Sewer mains
        arcpy.SelectLayerByLocation_management("SewerMain_Layer2", "INTERSECT", "TMK_Layer2")
        intersects_mains = arcpy.Describe("SewerMain_Layer2").FIDSet  # Get intersected features
        row[1] = "Yes" if intersects_mains else "No"
        
        # Check for intersection with Cesspools
        arcpy.SelectLayerByLocation_management("Cesspool_Layer2", "INTERSECT", "TMK_Layer2")
        intersects_cesspools = arcpy.Describe("Cesspool_Layer2").FIDSet
        row[2] = "Yes" if intersects_cesspools else "No"
        
        # Update the row
        cursor.updateRow(row)

# Clear selections
arcpy.SelectLayerByAttribute_management("TMK_Layer2", "CLEAR_SELECTION")
arcpy.SelectLayerByAttribute_management("SewerMain_Layer2", "CLEAR_SELECTION")
arcpy.SelectLayerByAttribute_management("Cesspool_Layer2", "CLEAR_SELECTION")

print("Analysis complete. Fields updated!")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Layer: Dataset TMK_Layer2 already exists.
Failed to execute (MakeFeatureLayer).
